# Set Required Args to Activate Modules

In [1]:
import RES.RESources as RES

 └> Using CODERS API key: w0OksqQKMFMtKPbO


In [2]:
vis={}

In [3]:
# Iterate over provinces for both solar and wind resources
resource_types = ['wind','solar']  #
provinces=['QC']  #,'AB','SK','ON','NS','MB'
for province_code in provinces:
    for resource_type in resource_types:
        required_args = {
            "config_file_path": 'config/config_CAN.yaml',
            "region_short_code": province_code,
            "resource_type": resource_type
        }
        
        # Create an instance of Resources and execute the module
        RES_module = RES.RESources_builder(**required_args)
        # RES_module.build(select_top_sites=True,
        #                  use_pypsa_buses=False)
        

2025-07-03 07:58:28,071 - INFO - Directory 'data/downloaded_data/NREL/ATB/ATBe.parquet' found locally.


____________________________________________________________
     Initiating RESource Builder...
____________________________________________________________
 └> NREL_ATBProcessor initiated...
  └> Processing Annual Technology Baseline (ATB) data sourced from NREL...
  └> ATB cost datafile: ATBe.parquet loaded
  └> Extracting technology baseline costs...
  └─> Extracting Solar PV technology cost...
>> Data (GeoDataFrame/DataFrame) saved to data/store/resources_QC.h5 with key 'cost/atb/solar'
  └─> Extracting Wind Turbine technology cost...
>> Data (GeoDataFrame/DataFrame) saved to data/store/resources_QC.h5 with key 'cost/atb/wind'
  └─> Extracting BESS technology cost...


2025-07-03 07:58:28,647 - INFO - Directory 'data/downloaded_data/NREL/ATB/ATBe.parquet' found locally.


>> Data (GeoDataFrame/DataFrame) saved to data/store/resources_QC.h5 with key 'cost/atb/bess'
 └> Snapshot for Resources: 2023-01-01 07:00:00 to 2024-01-01 06:00:00
____________________________________________________________
     Initiating RESource Builder...
____________________________________________________________
 └> NREL_ATBProcessor initiated...
  └> Processing Annual Technology Baseline (ATB) data sourced from NREL...
  └> ATB cost datafile: ATBe.parquet loaded
  └> Extracting technology baseline costs...
  └─> Extracting Solar PV technology cost...
>> Data (GeoDataFrame/DataFrame) saved to data/store/resources_QC.h5 with key 'cost/atb/solar'
  └─> Extracting Wind Turbine technology cost...
>> Data (GeoDataFrame/DataFrame) saved to data/store/resources_QC.h5 with key 'cost/atb/wind'
  └─> Extracting BESS technology cost...
>> Data (GeoDataFrame/DataFrame) saved to data/store/resources_QC.h5 with key 'cost/atb/bess'
 └> Snapshot for Resources: 2023-01-01 07:00:00 to 2024-01-0

In [ ]:
data1=RES_module.get_grid_cells()

2025-07-03 07:58:29,154 - INFO - >> Loading GADM data for Canada from local datafile data/downloaded_data/GADM/gadm41_Canada_L2.geojson.


 └> Preparing Grid Cells...


2025-07-03 07:58:29,864 - INFO - Created 99 records
2025-07-03 07:58:29,865 - INFO - GADM data for Québec saved to data/processed_data/regions/gadm41_Canada_L2_QC.geojson.
2025-07-03 07:58:29,866 - INFO - Building new cutout data/downloaded_data/cutout/QC_2023_2024.nc
2025-07-03 07:58:29,891 - INFO - Storing temporary files in /tmp/tmpu3poslh_
2025-07-03 07:58:29,894 - INFO - Calculating and writing with module era5:
2025-07-03 07:58:29,950 - INFO - Requesting data for feature influx...
2025-07-03 07:58:29,951 - INFO - Requesting data for feature wind...
2025-07-03 07:58:29,953 - INFO - Requesting data for feature runoff...
2025-07-03 07:58:29,955 - INFO - Requesting data for feature height...
2025-07-03 07:58:29,958 - INFO - Requesting data for feature temperature...


In [ ]:

data2=RES_module.get_cell_capacity()

In [ ]:
data3=RES_module.extract_weather_data()

In [ ]:
data4=RES_module.update_gwa_scaled_params() # testing, 2025 04 21

In [ ]:
data5=RES_module.get_CF_timeseries()

In [ ]:
data6=RES_module.find_grid_nodes()

In [ ]:
data7=RES_module.score_cells()

In [ ]:
data8=RES_module.get_clusters()
data9=RES_module.get_cluster_timeseries()
data10=RES_module.units.create_units_dictionary()

In [ ]:
vis['land_avail_1']=RES_module.cell_processor.plot_ERAF5_grid_land_availability()
vis['land_avail_2']=RES_module.cell_processor.plot_excluder_land_availability()
for raster_type in RES_module.cell_processor.raster_types:
            vis[f'land_vis_gaez_raster_{raster_type['name']}']=RES_module.cell_processor.__clip_to_boundary_n_plot__(raster_type, RES_module.cell_processor.region_boundary.geometry,True)

# Explore the outputs from Store

In [ ]:
from RES.hdf5_handler import DataHandler
res_store=DataHandler(f'data/store/resources_{province_code}.h5')

In [ ]:
cells=res_store.from_store('cells')
boundary=res_store.from_store('boundary')
solar_clusters=res_store.from_store('clusters/solar')
wind_clusters=res_store.from_store('clusters/wind')
solar_clusters_ts=res_store.from_store('timeseries/clusters/solar')
wind_clusters_ts=res_store.from_store('timeseries/clusters/wind')

In [ ]:
# wind_clusters[wind_clusters['lcoe']<=100].explore('potential_capacity')

# Playground for Top Site Selection

In [ ]:
resource_clusters_solar,cluster_timeseries_solar=RES_module.select_top_sites(solar_clusters,
                                                                solar_clusters_ts,
                                                                    resource_max_capacity=10)

resource_clusters_wind,cluster_timeseries_wind=RES_module.select_top_sites(wind_clusters,
                                                                wind_clusters_ts,
                                                                    resource_max_capacity=50)

In [ ]:
RES_module.export_results('wind',
                    resource_clusters_wind,
                    cluster_timeseries_wind,)

In [ ]:
RES_module.export_results('solar',
                    resource_clusters_solar,
                    cluster_timeseries_solar,)

In [ ]:
# resource_clusters_solar.plot('potential_capacity',legend=True)
# resource_clusters_wind.plot('potential_capacity',legend=True)

# Visualize Results

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

legend_x_ax_offset=1
# Ensure 'Region' is in the columns for both boundary and cells
if 'Region' not in boundary.columns:
    boundary = boundary.reset_index(inplace=True)

# Assign a number to each region
boundary['Region_Number'] = range(1, len(boundary) + 1)

# Define custom bins and labels for solar and wind capacity
solar_bins = [0, 100, 200, 300, 500, float('inf')]  # Custom ranges
solar_labels = ['<100','100-200', '200-300', '300-500','>500']  # Labels for legend

# Define custom bins and labels for solar and wind capacity
wind_bins = [0, 300, 500, 1000, 2000,3000, float('inf')]  # Custom ranges
wind_labels = ['<300','300-500', '500-1000', '1000-2000','2000-3000', '>3000']  # Labels for legend

# Categorize potential_capacity_solar and potential_capacity_wind into bins
resource_clusters_solar['solar_category'] = pd.cut(resource_clusters_solar['potential_capacity'], bins=solar_bins, labels=solar_labels, include_lowest=True)
resource_clusters_wind['wind_category'] = pd.cut(resource_clusters_wind['potential_capacity'], bins=wind_bins, labels=wind_labels, include_lowest=True)

# Create figure and axes for side-by-side plotting
fig, (ax1, ax2) = plt.subplots(figsize=(18, 8), ncols=2)
fig.suptitle("Potential Sites for Targeted Capacity Investments", fontsize=16,weight='bold')
# Set axis off for both subplots
ax1.set_axis_off()
ax2.set_axis_off()

# Shadow effect offset
shadow_offset = 0.01

# Plot solar map on ax1
# Add shadow effect for solar map
boundary.geometry = boundary.geometry.translate(xoff=shadow_offset, yoff=-shadow_offset)
boundary.plot(ax=ax1, color='None', edgecolor='grey', linewidth=1, alpha=0.7)  # Shadow layer
boundary.geometry = boundary.geometry.translate(xoff=-shadow_offset, yoff=shadow_offset)

# Plot solar cells
resource_clusters_solar.plot(column='solar_category', ax=ax1, cmap='Wistia', legend=True, 
           legend_kwds={'title': "Solar Capacity (MW)", 'loc': 'upper right','bbox_to_anchor':(legend_x_ax_offset,1), 'frameon': False})

# Plot actual boundary for solar map
boundary.plot(ax=ax1, facecolor='none', edgecolor='black', linewidth=0.2, alpha=0.7)
"""
# Annotate region numbers for solar map
for idx, row in boundary.iterrows():
    centroid = row.geometry.centroid
    ax1.annotate(f"{row['Region_Number']}", 
                 xy=(centroid.x, centroid.y), 
                 ha='center', va='center',
                 fontsize=7, color='black',
                 bbox=dict(facecolor='white', edgecolor='none', alpha=0.7, boxstyle='round,pad=0.2'))
"""
# Plot wind map on ax2
# Add shadow effect for wind map
boundary.geometry = boundary.geometry.translate(xoff=shadow_offset, yoff=-shadow_offset)
boundary.plot(ax=ax2, color='None', edgecolor='grey', linewidth=1, alpha=0.7)  # Shadow layer
boundary.geometry = boundary.geometry.translate(xoff=-shadow_offset, yoff=shadow_offset)

# Plot wind cells
resource_clusters_wind.plot(column='wind_category', ax=ax2, cmap='summer', legend=True, 
           legend_kwds={'title': "Wind Capacity (MW)", 'bbox_to_anchor':(legend_x_ax_offset,1), 'frameon': False})

# Plot actual boundary for wind map
boundary.plot(ax=ax2, facecolor='none', edgecolor='black', linewidth=0.2, alpha=0.7)
"""
# Annotate region numbers for wind map
for idx, row in boundary.iterrows():
    centroid = row.geometry.centroid
    ax2.annotate(f"{row['Region_Number']}", 
                 xy=(centroid.x, centroid.y), 
                 ha='center', va='center',
                 fontsize=8, color='black',
                 bbox=dict(facecolor='white', edgecolor='none', alpha=0.7, boxstyle='round,pad=0.2'))
"""
# Adjust layout for cleaner appearance
fig.patch.set_alpha(0)  # Make figure background transparent
plt.tight_layout()

# Show the side-by-side plot

# plt.savefig('solar_wind_capacity_map.png',dpi=300)
plt.show()